In [ ]:
import numpy as np 
import pandas as pd
from matplotlib import pyplot as plt 

In [ ]:
data = pd.read_csv('train.csv')

In [ ]:
data.head()

In [ ]:
data = np.array(data)
m, n = data.shape
np.random.shuffle(data)

data_dev = data[0:1000].T # Transpose the dev set
Y_dev = data_dev[0]
X_dev = data_dev[1:n] # X_dev will have 784 rows (features) and 1000 columns (samples)
X_dev = X_dev / 255.0 # Normalize pixel values

data_train  = data[1000:m].T # Transpose the training set
Y_train = data_train[0]
X_train = data_train[1:n] # X_train will have 784 rows and (m-1000) columns
X_train = X_train / 255.0 # Normalize pixel values


In [ ]:
def init_params():
    input_size_L1 = 784
    hidden_size_L1 = 10
    output_size_L2 = 10

    # He Initialization for W1: sqrt(2 / input_size_L1)
    # This helps maintain variance of activations when using ReLU
    W1 = np.random.randn(hidden_size_L1, input_size_L1) * np.sqrt(2. / input_size_L1)
    B1 = np.zeros((hidden_size_L1, 1))
    # He Initialization for W2: sqrt(2 / hidden_size_L1)
    # Note: For the output layer with Softmax, sometimes Xavier is preferred,
    # but He often works well too.
    W2 = np.random.randn(output_size_L2, hidden_size_L1) * np.sqrt(2. / hidden_size_L1)
    B2 = np.zeros((output_size_L2, 1))

    return W1, B1, W2, B2

def ReLU(Z):
    return np.maximum(0, Z)

def softMax(Z):
    exp_Z = np.exp(Z - np.max(Z, axis=0, keepdims=True)) 
    return exp_Z / np.sum(exp_Z, axis=0, keepdims=True)
    
def forward_prop(W1, B1, W2, B2, X, keep_prob=1.0, training=True): # Added keep_prob and training
    # Performs forward propagation with dropout.
    Z1 = W1.dot(X) + B1
    A1 = ReLU(Z1)

    D1 = None # Initialize D1
    if training and keep_prob < 1.0:
        # Create a dropout mask: randomly set some neurons to 0
        D1 = np.random.rand(*A1.shape) < keep_prob
        A1 = A1 * D1
        # Scale the activations by 1/keep_prob to maintain the expected sum of activations
        A1 = A1 / keep_prob

    Z2 = W2.dot(A1) + B2
    A2 = softMax(Z2)

    return Z1, A1, Z2, A2, D1 # Return D1 for backprop

def one_hot(Y):
    Y = np.array(Y) # Create a one-hot encoded matrix with shape (number of classes, number of samples)
    one_hot_Y = np.zeros((Y.max() + 1, Y.size)) # Set the appropriate elements to 1
    one_hot_Y[Y, np.arange(Y.size)] = 1
    return one_hot_Y

def deriv_ReLU(Z):
    return Z > 0

def back_prop(Z1, A1, Z2, A2, W1, W2, X, Y, D1, *, keep_prob=1.0, lambda_reg=0.01): 

    # Performs backpropagation with dropout and L2 regularization.
    m = X.shape[1]
    one_hot_Y = one_hot(Y)

    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T) + (lambda_reg / m) * W2 # L2 regularization
    db2 = 1 / m * np.sum(dZ2, axis=1, keepdims=True)

    dZ1 = W2.T.dot(dZ2) * deriv_ReLU(Z1)
    if D1 is not None and keep_prob < 1.0:
        dZ1 = dZ1 * D1
        dZ1 = dZ1 / keep_prob

    dW1 = 1 / m * dZ1.dot(X.T) + (lambda_reg / m) * W1 # L2 regularization
    db1 = 1 / m * np.sum(dZ1, axis=1, keepdims=True)

    return dW1, db1, dW2, db2


def update_params_adam(W1, B1, W2, B2, dW1, db1, dW2, db2, alpha, t, v_dW1, v_db1, v_dW2, v_db2, s_dW1, s_db1, s_dW2, s_db2, beta1=0.9, beta2=0.999, epsilon=1e-8):
    #Updates the parameters using the Adam optimization algorithm.
    
    # Update biased first moment estimates
    v_dW1 = beta1 * v_dW1 + (1 - beta1) * dW1
    v_db1 = beta1 * v_db1 + (1 - beta1) * db1
    v_dW2 = beta1 * v_dW2 + (1 - beta1) * dW2
    v_db2 = beta1 * v_db2 + (1 - beta1) * db2

    # Update biased second moment estimates
    s_dW1 = beta2 * s_dW1 + (1 - beta2) * (dW1**2)
    s_db1 = beta2 * s_db1 + (1 - beta2) * (db1**2)
    s_dW2 = beta2 * s_dW2 + (1 - beta2) * (dW2**2)
    s_db2 = beta2 * s_db2 + (1 - beta2) * (db2**2)

    # Correct bias for first moment estimates
    v_dW1_corrected = v_dW1 / (1 - beta1**t)
    v_db1_corrected = v_db1 / (1 - beta1**t)
    v_dW2_corrected = v_dW2 / (1 - beta1**t)
    v_db2_corrected = v_db2 / (1 - beta1**t)

    # Correct bias for second moment estimates
    s_dW1_corrected = s_dW1 / (1 - beta2**t)
    s_db1_corrected = s_db1 / (1 - beta2**t)
    s_dW2_corrected = s_dW2 / (1 - beta2**t)
    s_db2_corrected = s_db2 / (1 - beta2**t)

    # Update parameters
    W1 = W1 - alpha * v_dW1_corrected / (np.sqrt(s_dW1_corrected) + epsilon)
    B1 = B1 - alpha * v_db1_corrected / (np.sqrt(s_db1_corrected) + epsilon)
    W2 = W2 - alpha * v_dW2_corrected / (np.sqrt(s_dW2_corrected) + epsilon)
    B2 = B2 - alpha * v_db2_corrected / (np.sqrt(s_db2_corrected) + epsilon)

    return W1, B1, W2, B2, v_dW1, v_db1, v_dW2, v_db2, s_dW1, s_db1, s_dW2, s_db2


In [ ]:
def get_predictions(A2):
    predictions = np.argmax(A2, axis=0)
    return predictions

def get_accuracy(predictions, Y):
    Y = np.array(Y)
    # print(predictions, Y)
    return np.sum(predictions == Y) / Y.size
    

def gradient(X, Y, iterations, alpha, lambda_reg=0.01, keep_prob=0.8):
    W1, B1, W2, B2 = init_params()

    # Initialize Adam moment estimates
    v_dW1, v_db1 = np.zeros_like(W1), np.zeros_like(B1)
    v_dW2, v_db2 = np.zeros_like(W2), np.zeros_like(B2)
    s_dW1, s_db1 = np.zeros_like(W1), np.zeros_like(B1)
    s_dW2, s_db2 = np.zeros_like(W2), np.zeros_like(B2)

    for i in range(1, iterations + 1):
        # Pass keep_prob and training=True to forward_prop
        Z1, A1, Z2, A2, D1 = forward_prop(W1, B1, W2, B2, X, keep_prob=keep_prob, training=True)
        # Pass W1, D1, keep_prob, and lambda_reg to back_prop (W1 added here!)
        dW1, db1, dW2, db2 = back_prop(Z1, A1, Z2, A2, W1, W2, X, Y, D1, keep_prob=keep_prob, lambda_reg=lambda_reg)

        W1, B1, W2, B2, v_dW1, v_db1, v_dW2, v_db2, s_dW1, s_db1, s_dW2, s_db2 = \
            update_params_adam(W1, B1, W2, B2, dW1, db1, dW2, db2, alpha, i, v_dW1, v_db1, v_dW2, v_db2, s_dW1, s_db1, s_dW2, s_db2)

        if i % 500 == 0:
            print(f"Iteration: {i}")
            predictions = get_predictions(A2)
            accuracy = get_accuracy(predictions, Y)
            print(f"Accuracy: {accuracy:.4f}")

    return W1, B1, W2, B2 

In [ ]:
W1, B1, W2, B2 = gradient(X_train, Y_train, 2000, 0.01)

In [ ]:
def make_predictions (X, W1, b1, W2, b2):
    _, _, _, A2, _ = forward_prop(W1, b1, W2, b2, X, keep_prob=1.0, training=False)
    predictions = get_predictions (A2)
    return predictions

def test_prediction (index, W1, b1, W2, b2):
    current_image = X_train[:, index, None]
    prediction = make_predictions (X_train[:, index, None], W1, b1, W2, b2)
    label = Y_train[index]
    print("Prediction:", prediction)
    print("Label: ", label)
    current_image = current_image.reshape((28, 28)) * 255
    plt.gray()
    plt.imshow(current_image, interpolation='nearest')
    plt.show()

In [ ]:
dev_predictions = make_predictions(X_dev, W1, B1, W2, B2)
get_accuracy(dev_predictions, Y_dev)

In [ ]:
test_prediction(70, W1, B1, W2, B2)  # Test with the image in the training set